In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Sales_order_Query='''


with original
as 
(SELECT 
min(dly_ord_itm_actvy_ts) as first_add,
bus_prod_instnc_id
 FROM `cio-datahub-enterprise-pr-183a.ent_cust_ord_actvy.bq_dly_wln_ord_item_actvy` WHERE ord_act_typ_cd = 'Add' and prod_typ_cd in ('SMHM') and ord_act_stat_cd = 'Processed' and bus_prod_instnc_id is not null
 --and date(dly_ord_itm_actvy_ts) >= '2023-10-01' and date(dly_ord_itm_actvy_ts) < '2024-01-01'

 group by bus_prod_instnc_id) /* Fetch original order date for the product instance */



,BAN_list as (

SELECT CAST(BAN as STRING) as BAN_data
 --FROM `pras-pr-223186.pras_pr_dataset.Outbound_Robin` 
 FROM `pras-pr-223186.pras_pr_dataset.Outbound_Robin_08Mar2024`

)



,details as
(
SELECT 
dly_ord_itm_actvy_ts,
bus_billg_acct_num,
bus_prod_instnc_id,
prod_nm,
munic_nm,
prov_state_cd,
bi_chnl_tag_cd,
chnl_org_id,
prod_typ_cd,
src_sls_rep_cd,
src_typ_cd,
src_usr_chnl_txt,
txn_sub_typ_txt,
SLS_ACTVY_TXT
 FROM `cio-datahub-enterprise-pr-183a.ent_cust_ord_actvy.bq_dly_wln_ord_item_actvy` WHERE ord_act_typ_cd = 'Add' 
 --and prod_typ_cd in ('SMHM','HSIC','SING','TTV') 
 and prod_typ_cd in ('SMHM')
 and ord_act_stat_cd = 'Processed'
 --and date(dly_ord_itm_actvy_ts) >= '2023-10-01' and date(dly_ord_itm_actvy_ts) < '2024-01-01' 
 
 )
 
 
 
  /* Gather channel details */

,channel_refs as (


with
latest_update as (
SELECT
chnl_org_id,
max(chnl_org_key) as latest_key FROM `cio-datahub-enterprise-pr-183a.ent_sls_chnl.bq_channel_org_dim`
group by chnl_org_id

)

select distinct
t1.chnl_org_id,
chnl_org_txt from `cio-datahub-enterprise-pr-183a.ent_sls_chnl.bq_channel_org_dim` t1 inner join latest_update on latest_key = chnl_org_key

)

,Sales_Order_DF as (

select
CAST(details.bus_billg_acct_num as STRING) as BAN_sales_order,
date(details.dly_ord_itm_actvy_ts) as Order_date,
details.bus_billg_acct_num as BAN,
cast (details.bus_prod_instnc_id as Integer) as Prod_Instnc_ID,
details.prod_nm as SMHM_Plan_Name,
details.munic_nm as Municpality_Name,
details.prov_state_cd as State_Code,
details.bi_chnl_tag_cd as Sales_Channel_tag,
details.chnl_org_id,
channel_refs.chnl_org_txt as  Sales_Dealer_Name,
details.prod_typ_cd as Prod_Type,
details.src_sls_rep_cd as Sales_Agent_ID,
details.src_typ_cd,
details.src_usr_chnl_txt,
details.txn_sub_typ_txt,
details.SLS_ACTVY_TXT
from details inner join original on original.bus_prod_instnc_id = details.bus_prod_instnc_id and original.first_add = details.dly_ord_itm_actvy_ts  /* Inner join them to prevent duplicates */
 left join channel_refs on details.chnl_org_id = channel_refs.chnl_org_id /* left joining to get channel dealer names */
 where bi_chnl_tag_cd <> 'UNKNOWN' 
 order by details.dly_ord_itm_actvy_ts
)


,Sales_order_data_rpt as (



  SELECT
 bill_account_number  AS BAN_sales,
  reporting_dt AS order_created_date,
  activation_dt AS activation_date,
  platform,
  product_family,
  channel_group,
  sales_agent_id,
  current_sales_agent_id,
initial_sales_agent_id,
  sales_agent_nm,
  product_nm as Plan_name_1
FROM
  `bi-srv-hsmsd-3c-pr-ca2cd4.hsmsd_3c_rpt_dataset.bq_rpt_chnl_order_ffh_dtl_view`
WHERE
  is_gross_new_order = 1
  AND is_initial_order = 1
  AND product_family IN ('SMHM')

) 


select * 
from BAN_list a
left join Sales_Order_DF b
on a.BAN_data=b.BAN_sales_order 
left join Sales_order_data_rpt rpt
on a.BAN_data=rpt.BAN_sales

'''

In [ ]:
DF_sales_Order=extract_bq_data(bq_client, sql=Sales_order_Query)

In [ ]:
DF_sales_Order.info()

In [ ]:
DF_sales_Order['Sales_Agent_ID'].value_counts()

In [ ]:
DF_sales_Order['Sales_Agent_ID'].replace('UKN', np.nan, inplace=True)

In [ ]:
# Columns in order of priority to fill the nulls
replacement_cols = ['Sales_Agent_ID', 'sales_agent_id_1', 'current_sales_agent_id','initial_sales_agent_id','sales_agent_nm']

# Create a new column 'E' that starts as all NaN
DF_sales_Order['Sales_Agent_Final'] = pd.NA

# Iteratively fill 'E' using 'fillna()' from columns A, B, and C
for col in replacement_cols:
    DF_sales_Order['Sales_Agent_Final'] = DF_sales_Order['Sales_Agent_Final'].fillna(DF_sales_Order[col])

DF_sales_Order['Sales_Agent_Final'].value_counts()

In [ ]:
DF_sales_Order['SMHM_Plan_Name'].value_counts()

In [ ]:
DF_sales_Order['SMHM_Plan_Name_Final'] = DF_sales_Order['SMHM_Plan_Name'].fillna(DF_sales_Order['Plan_name_1'])
DF_sales_Order['SMHM_Plan_Name_Final'].value_counts()

In [ ]:
DF_sales_Order['Sales_Channel_tag'].value_counts()

In [ ]:
DF_sales_Order['SMHM_Channel_Final'] = DF_sales_Order['Sales_Channel_tag'].fillna(DF_sales_Order['channel_group'])
DF_sales_Order['SMHM_Channel_Final'].value_counts()

In [ ]:
DF_sales_Order.info()

In [ ]:
DF_sales_Order.to_csv('Outbound_Robin_with_Sales_08Mar2024.csv',index=False)